<a href="https://colab.research.google.com/github/KendallScott/QTW/blob/main/Case%20Study%203/Case_Study_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [344]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [359]:
file_list = []
for root, dirs, files in os.walk("/content/drive/MyDrive/SpamAssassinMessages/", topdown=False):
    for name in files:
        tmp = os.path.join(root,name)
        file_list.append(tmp)
    for item in dirs:
        print(item)

file_list_full = []
for i in range(9352):
    file_list_full.append(file_list[i])

spam_2
spam
easy_ham_2
easy_ham
hard_ham


Option one-- sets the decoder dynamically, but still needs more work for pulling in information on other email types


In [369]:
import quopri
import codecs
import magic
import email
from gensim.parsing.preprocessing import STOPWORDS
from nltk import word_tokenize
import chardet

i=0


contents = []
types = []
labels = []

for root, dirs, files in os.walk("/content/drive/MyDrive/SpamAssassinMessages/"):
    for name in files:
      
        #this may need to be adjusted-- 

        blob = open(file_list_full[i], 'rb').read()

        #determining the encoding of the email, so the open(os.path) portion doesn't hit an error downloading an email

        result = chardet.detect(blob)
        encoding_dynamic = result['encoding']

        if 'unknown' in encoding_dynamic:
          #this needs to be setup to pull in the correct encoding when magic fails to identify the right encoding
          encoding_dynamic='utf-8'

        with open(os.path.join(root,name), encoding=encoding_dynamic, errors='replace') as f:
            x = email.message_from_file(f)
            print('normal', file_list_full[i])
            i+=1

            if "multipart" in x.get_content_type():
            # Clean up multipart email to raw string
              content_parts = []

              for part in x.walk():
                if part.get_content_type() == "text/plain":
                      content = part.get_payload(decode=True).decode("utf-8").replace("\n", " ")
                      content_parts.append(content)
                content = "\n".join(content_parts)
                contents.append(content)

            elif "text/plain" in x.get_content_type():  #boy I hope all text messages are the same!
                # clean up multipart email to raw string
                contents.append(x.get_payload().replace("\n"," "))

            elif "html" in x.get_content_type():
                # clean up html email to raw string
                contents.append(x)
                types.append(x.get_content_type())

            if "ham" in root:
                labels.append(0)
            elif "spam" in root:
                labels.append(1)
            else:
                print("YOU HAVE A BIG PROBLEM--LABEL NOT FOUND")
                
            if "ham" in root:
                labels.append(0)
            elif "spam" in root:
                labels.append(1)
            else:
                print("YOU HAVE A BIG PROBLEM--LABEL NOT FOUND")
                
        

normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00553.4e22bb923ee41a61d04f8b275157366b
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00724.edc5a3ce2ea91fedbf00aa0c30459a87
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00585.0cc56d33bcfde91ab75bf202e4684c4a
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00653.dcb006e0c0aaa7aa3e5bd7cb3d444f23
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00484.602c7afb217663a43dd5fa24d97d1ca4
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00591.962cc31322a42abd7ca205b62c56438e
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00543.e69bd0a0effd4a12537fb358d79ea337
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00463.0bc4e08af0529dd773d9f10f922547db
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00458.49467dba97d6ac2825c533df9a7a5017
normal /content/drive/MyDrive/SpamAssassinMessages/spam_2/00519.f189e2f1541968e48de6ebd9db23b35d
normal /content/drive/MyDrive/

UnicodeDecodeError: ignored

In [376]:
import pandas as pd 

data = pd.DataFrame({"Text":contents,"Label":labels})
data

,Text,Label
0,How would you like a Top Rated Law Firm workin...,1
1,SUBJECT: Urgent Concern Pl...,1
2,"[Return-Path, Delivery-Date, Received, Receive...",1
3,"As seen on NBC, CBS, CNN, and even Oprah! The ...",1
4,,1
...,...,...
9349,"Hi XFS developers, When studying xfs code(2.4....",0
9350,My gut reaction would be to tell them to go an...,0
9351,* WORD A DAY * ---------...,0
9352,==============================================...,0


In [ ]:
len(contents)

In [ ]:
len(types)

In [ ]:
len(labels)

In [ ]:
import pandas as pd 

data = pd.DataFrame({"Text":contents,"Label":labels})

In [ ]:
data[data['type'].str.contains("html")]

In [ ]:
data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
out = vectorizer.fit_transform(data['Text'])

In [ ]:
vectorizer.vocabulary_

In [ ]:
data.iloc[0,0]

In [ ]:
<unk>
            

In [ ]:
reverse={}
for word,value in vectorizer.vocabulary_.items():
    reverse[value]=word

In [ ]:
reverse[64792]

In [ ]:
for i in range(out[0].shape[1]):
    if out[0,i]>0:
        print(reverse[i], out[0,i])

In [ ]:
out[0,0]

In [ ]:
vectorizer.vocabulary_['unk']

In [ ]:
<unk> = 102
<start> = 101
<end>= 3
<pad> = 100
<cls>

In [ ]:
"<pad> <pad> <start> The <unk> car drove away <end>"

In [ ]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_counts = CountVectorizer()
v_data = word_counts.fit_transform(data['data'])
v_test = word_counts.transform(test['data'])

In [ ]:
v_data

In [ ]:
from sklearn.naive_bayes import ComplementNB, BernoulliNB, GaussianNB, CategoricalNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
comp = ComplementNB()
bern = BernoulliNB()
gaus = GaussianNB()
cat = CategoricalNB()
multi = MultinomialNB()
methods = {"Complement":comp,"Bernoulli":bern,"Gausian":gaus,"Multi":multi,"Cat":cat}
for i in methods.keys():
    try:
        methods[i].fit(v_data,data['target'])
        p = methods[i].predict(v_test)
        print(accuracy_score(test['target'],p), f"method={i}")
        print(classification_report(test['target'],p))
    except:
        print(f"{i} failed")


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)
ConfusionMatrixDisplay.from_predictions(test['target'],p,cmap='Blues')

In [ ]:
reverse={}
for word,value in word_counts.vocabulary_.items():
    reverse[value]=word